In [ ]:
def choose_action(e, state) -> tuple[int, int]:
    pass


def train():
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    states, actions, rewards, dones, next_states = zip(*batch)

    states = torch.stack(states).to(device)
    actions = torch.tensor(actions, dtype=torch.int64, device=device).reshape(batch_size)
    rewards = torch.tensor(rewards, dtype=torch.float32).reshape(batch_size).to(device)  # Convert rewards to a tensor
    dones = torch.tensor(dones, dtype=torch.float32).reshape(batch_size).to(device)  # Convert dones to a tensor

    q_values = model(states)
    predicted_q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # get the q values for the corresponding actions taken

    next_q_values = torch.zeros(batch_size, device=device)

    non_terminal_mask = (dones == 0)  # Mask for non-terminal states
    non_terminal_next_states = [s for s in next_states if s is not None]

    if non_terminal_next_states:
        non_terminal_next_states = torch.stack(non_terminal_next_states).to(device)
        next_q_values[non_terminal_mask] = torch.max(target_network(non_terminal_next_states), dim=1).values

    target_q_values = rewards + gamma * next_q_values * (1 - dones)  # target q values according to bellman equation

    loss = criterion(target_q_values, predicted_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


for episode in range(1, 500_001):
    if episode % 100 == 0:
        epsilon = max(epsilon * 0.99, 0.05)  # decay the exploration chance

    game = Game()  # each episode starts a new game; the deck is NOT re-used.
    while not game.is_over:
        state = get_game_state(game)
        action = choose_action(epsilon, state)

        game.take_action(action)

        reward = game.score
        done = int(game.is_over)

        if done:
            next_state = None
        else:
            next_state = get_game_state(game)

        memory.append((state, action, reward, done, next_state))

    train()